## Library imports

In [ ]:
import os
import gc
import json 
import glob
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# plotting library
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt

from PIL import Image
import cv2

# interactive plots
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings  
warnings.filterwarnings('ignore')

## Helper functions

In [ ]:
def getFileData(file):    
    wifi = list()
    waypoint = list()
    with open(file) as f:
        txt = f.readlines()
    for line in txt:
        line = line.strip().split()
        if line[1] == "TYPE_WAYPOINT":
            waypoint.append(line)
        if line[1] == "TYPE_WIFI":
            wifi.append(line)
    
    # WiFi dataframe
    wiFiDf = pd.DataFrame(np.array(wifi))
    # sort by time
    wiFiDf.sort_values(by=[0],inplace=True)

    # waypoint df
    waypointDf = pd.DataFrame(np.array(waypoint))
    waypointDf.sort_values(by=[0], inplace=True)    

    return wiFiDf, waypointDf

In [ ]:
def getPointData(wiFiDf, waypointDf):        
    wayPtTimestamps = waypointDf[0].values.astype(np.int64)
    wifiTimeStamps = wiFiDf[0].values.astype(np.int64)
    
    waypointx = waypointDf[2].values.astype(float)
    waypointy = waypointDf[3].values.astype(float)
    waypointData = np.column_stack((waypointx, waypointy))
        
    wifiAPx = np.interp(wifiTimeStamps, wayPtTimestamps, waypointx)
    wifiAPy = np.interp(wifiTimeStamps, wayPtTimestamps, waypointy)
    wifiAPData = np.column_stack((wifiAPx, wifiAPy))

    return wayPtTimestamps, waypointData, wifiTimeStamps, wifiAPData

```python
def getGeoJsonInfo(jsonFilePath):

    # Opening JSON file 
    f = open(inputFile)

    # returns JSON object as  a dictionary 
    geoJsonData = json.load(f) 
    
    boundaryPoints = []
    for idx,feature in enumerate(geoJsonData['features']):
        boundaryPoints.append(feature['geometry']['coordinates'])
    print(np.squeeze(np.array(feature['geometry']['coordinates']), axis=0).shape)
    boundaryPoints = np.squeeze(np.concatenate(boundaryPoints,axis=1), axis=0)
    print(boundaryPoints.shape)
    return boundaryPoints

# inputFile = 'metadata/5a0546857ecc773753327266/B1/geojson_map.json'
# pts = getGeoJsonInfo(inputFile)
# plt.plot(pts[:,0], pts[:,1], 'ko')
```

In [ ]:
base_path = '../input/indoor-location-navigation'

# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

## Sample building site and floor

In [ ]:
building = used_buildings[0]
folders = sorted(glob.glob(os.path.join(base_path,'train', building +'/*')))
folder = folders[0]
floor = folder.split('/')[-1]
files = glob.glob(os.path.join(folder, "*.txt"))

print(f"building = {building}")
print(f"folder = {folder}")
print(f"floor = {floor}")

## Get interpolated waypoints acoridng to wifi timestamps

In [ ]:
interpolatedInfo = {}

for i,file in enumerate(files):
    fileName = file.split('/')[-1].rstrip('.txt')
    interpolatedInfo[fileName] = {}
    
    # get wifi and waypoint data from each path file
    wifiDf, waypointDf = getFileData(file)

    # linear interpolate x,y coordinates based on wifi timestamp
    wayPtTimestamps, waypointData, wifiTimeStamps, wifiAPData =  getPointData(wifiDf, waypointDf)

    # store output
    interpolatedInfo[fileName]["wayPtTimestamps"] = wayPtTimestamps
    interpolatedInfo[fileName]["waypointData"] = waypointData
    interpolatedInfo[fileName]["wifiTimeStamps"] = wifiTimeStamps
    interpolatedInfo[fileName]["wifiAPData"] = wifiAPData

In [ ]:
with open(f"{base_path}/metadata/{building}/{floor}/floor_info.json") as f:
    content = f.read()
    floor_info = json.loads(content)

print(f"floor_info = {floor_info}")

## Plot original data vs interpolated data

```python
img = plt.imread(f"{base_path}/metadata/{building}/{floor}/floor_image.png")    
plt.figure(figsize=(24,24))
plt.imshow(img, extent=[0,floor_info["map_info"]["width"],0,floor_info["map_info"]["height"]])

for pathFile in list(interpolatedInfo.keys()):
    pathData = interpolatedInfo[pathFile]
    #plt.plot(pathData["waypointData"][:,0], pathData["waypointData"][:,1],  label = f'{pathFile}_waypt')
    plt.plot(pathData["wifiAPData"][:,0], pathData["wifiAPData"][:,1], label = f'{pathFile}_interpolated',
             marker ='o', markersize=1)
#plt.legend(loc='best')
plt.grid(True)
plt.savefig(f"{pathFile}.png",dpi=300)
```

In [ ]:
img = cv2.imread(f"{base_path}/metadata/{building}/{floor}/floor_image.png").astype(np.uint8)
image = Image.fromarray(img)
layout = go.Layout(images=[dict(
                    source=image,
                    xref = 'x', yref='y', x=0, y =floor_info["map_info"]["height"],
                    sizex=floor_info["map_info"]["width"], sizey=floor_info["map_info"]["height"],
                    sizing= "stretch", opacity = 0.25, layer="below")])

fig = go.Figure(layout=layout)
for pathFile in list(interpolatedInfo.keys()):
    pathData = interpolatedInfo[pathFile]
    trace0 = go.Scatter(x=pathData["wifiAPData"][:,0], y=pathData["wifiAPData"][:,1],
                    name=f'{pathFile}_interpolated')
    trace1 = go.Scatter(x=pathData["waypointData"][:,0], y=pathData["waypointData"][:,1],
                    name=f'{pathFile}_waypt')
    fig.add_traces([trace0, trace1])
    
fig.show()   